<center>
<h1><br/></h1>
<h1>INF 582: Introduction To Text Mining And NLP</h1>
<h2>Challenge: News Articles Title Generation</h2>
<h3>Notebook: Grammar correction of the text in the news</h3>
<h4>Students Name: <br>
<b>DABERE Abasse<br>
FUERTES PANIZO Cynthia Yacel</b> </h4>
<br>
</center>

During our exploratory analysis, we discovered instances of sentences within the text lacking spaces in the training dataset. For instance, in the phrase "Un membre incontournableEt ce n'est autre," we observed a missing space between "incontournable" and "Et," which should be "incontournable Et." Consequently, we aim to enhance the grammar.

## `a.` Installing the libraries

In [ ]:
pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=f3fe1b3438baa60f5c4b58881d01a6a2fd3c98d008a5b32bd0550feb43c3db5b
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 30.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
pip install language-tool-python requests

## `b.` Methods that were attempted (prior to achieving successful outcomes) but did not yield favorable results.

### `b.1.` Using <i>import re</i>

In [ ]:
import re

def fix_spaces(text):
    corrected_text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    return corrected_text

In [ ]:
fix_spaces("Parisest lacapitale de laFrance")

'Parisest lacapitale de la France'

We can observe that using regex is not an efficient method for properly separating the words.

### `b.2.` Using <i>import spacy</i>

In [ ]:
import spacy
import fr_core_news_sm

def correct_spacing(text):
    nlp = spacy.load("fr_core_news_sm")
    doc = nlp(text)
    corrected_text = " ".join([token.text_with_ws for token in doc])
    return corrected_text

In [ ]:
correct_spacing("Parisest lacapitale de laFrance")

'Parisest  lacapitale  de  laFrance'

Using `spacy` for grammar correction proves ineffective as it fails to insert any spaces in our example.

### `b.3.` Using <i>import wordninja</i>

In [ ]:
import wordninja

def correct_wordninja(text):
    words = wordninja.split(text)
    corrected_text = " ".join(words)
    return corrected_text

In [ ]:
correct_wordninja("Parisest lacapitale de laFrance")

'Paris est la capitale de la France'

In [ ]:
df = pd.read_csv("data/train.csv")
df['corrected_news'] = df['text'].apply(correct_wordninja)
df.to_csv("data/train_spaceChecked.csv", index=False)

# Calculate how many news articles were corrected and their percentage
total_news = len(df)
corrected_news_count = df['corrected_news'].notna().sum()
percentage_corrected = (corrected_news_count / total_news) * 100

print("Number of news articles corrected:", corrected_news_count)
print("Percentage of news articles corrected:", percentage_corrected)

Number of news articles corrected: 21401
Percentage of news articles corrected: 100.0


In [ ]:
df

,text,titles,corrected_news
0,Thierry Mariani sur la liste du Rassemblement ...,L'information n'a pas été confirmée par l'inté...,Thierry Mariani sur la list e du Rassemblement...
1,C'est désormais officiel : Alain Juppé n'est p...,Le maire de Bordeaux ne fait plus partie des R...,C'est d so rma is of fi ciel Alain Jupp n ' es...
2,La mesure est décriée par les avocats et les m...,"En 2020, les tribunaux d'instance fusionnent a...",La mesure est d cri e par les avoca ts et les ...
3,Dans une interview accordée au Figaro mercredi...,"Les médecins jugés ""gros prescripteurs d'arrêt...",Dans une interview accord e au Figaro mer c re...
4,Le préjudice est estimé à 2 millions d'euros. ...,Il aura fallu mobiliser 90 gendarmes pour cett...,Le pr jud ice est es tim 2 millions d ' euros ...
...,...,...,...
21396,"Une ""main tendue"" que refuse de saisir l'eurod...",POLITIQUE. La présidente du Front national a s...,Une main ten due que refuse de sai sir l ' eur...
21397,"Venue présenter son dernier spectacle, Elektri...",Que serait un dimanche matin sans un passage e...,Venue pr s enter son dernier spectacle Ele kt ...
21398,Mauvaise nouvelle pour les amateurs de chasse ...,"La chasse à la glu sera interdite cette année,...",Mau v aise nouvelle pour les amateurs de chass...
21399,"""Ce n'est pas une question de simple voile, c'...","Bruno Le Maire, candidat à la primaire de la d...",Ce n ' est pas une question de simple voile c'...


In the begining, it seems like `import wordninja` properly separates the words from "ParisestlacapitaledelaFrance" to "Paris est la capital edel a France". Therefore, we applied this function across the training dataset. However, this approach yields unsatisfactory results when applied to the entire training dataset. For example, "La mesure est décriée par" is converted to "La mesure est d cri e par", which is clearly incorrect.

### `b.4.` Using <i>import nltk</i>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

def correct_nltk(text):
    words = word_tokenize(text)
    corrected_news = " ".join(words)
    return corrected_news

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
correct_nltk("Parisest lacapitale de laFrance")

'Parisest lacapitale de laFrance'

Using `nltk` for grammar correction proves ineffective as it fails to insert any spaces in our example.

## `c.` Method considered successful for correcting grammatical errors.

### `c.1.` Using <i>import language_tool_python</i>

In [ ]:
import pandas as pd
import language_tool_python
from tqdm import tqdm

# Initialize LanguageTool in French
tool = language_tool_python.LanguageTool('fr')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpcp70zgpo.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.2.zip to /root/.cache/language_tool_python.


In [ ]:
def correct_grammar(text):
    corrected_text = tool.correct(text)
    return corrected_text

In [ ]:
correct_grammar("Parisest lacapitale de laFrance")

'Paris est la capitale de la France'

It appears that using `language_tool_python.LanguageTool('fr')` for grammar correction is a promising approach. We'll now apply it across the entire training dataset to evaluate its effectiveness.

### `c.2.` Apply the function to the training set

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_train = "/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/train.csv"
df_train = pd.read_csv(path_train)
df_train

,text,titles
0,Thierry Mariani sur la liste du Rassemblement ...,L'information n'a pas été confirmée par l'inté...
1,C'est désormais officiel : Alain Juppé n'est p...,Le maire de Bordeaux ne fait plus partie des R...
2,La mesure est décriée par les avocats et les m...,"En 2020, les tribunaux d'instance fusionnent a..."
3,Dans une interview accordée au Figaro mercredi...,"Les médecins jugés ""gros prescripteurs d'arrêt..."
4,Le préjudice est estimé à 2 millions d'euros. ...,Il aura fallu mobiliser 90 gendarmes pour cett...
...,...,...
21396,"Une ""main tendue"" que refuse de saisir l'eurod...",POLITIQUE. La présidente du Front national a s...
21397,"Venue présenter son dernier spectacle, Elektri...",Que serait un dimanche matin sans un passage e...
21398,Mauvaise nouvelle pour les amateurs de chasse ...,"La chasse à la glu sera interdite cette année,..."
21399,"""Ce n'est pas une question de simple voile, c'...","Bruno Le Maire, candidat à la primaire de la d..."


In [ ]:
# Apply grammar correction to all news articles in the DataFrame
tqdm.pandas()
df_train['text_grammar'] = df_train['text'].progress_apply(correct_grammar)
file_path = "/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/train_grammar_correction.csv"
df_train.to_csv(file_path, index=False)

print("File saved to Google Drive at:", file_path)

100%|██████████| 21401/21401 [4:47:09<00:00,  1.24it/s]


File saved to Google Drive at: /content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/train_grammar_correction.csv


In [ ]:
import pandas as pd
path_train_grammar = "/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/train_grammar_correction.csv"
df_train_grammar = pd.read_csv(path_train_grammar)
df_train_grammar

,text,titles,text_grammar
0,Thierry Mariani sur la liste du Rassemblement ...,L'information n'a pas été confirmée par l'inté...,Thierry Marianne sur la liste du Rassemblement...
1,C'est désormais officiel : Alain Juppé n'est p...,Le maire de Bordeaux ne fait plus partie des R...,C'est désormais officiel : Alain Juppé n'est p...
2,La mesure est décriée par les avocats et les m...,"En 2020, les tribunaux d'instance fusionnent a...",La mesure est décriée par les avocats et les m...
3,Dans une interview accordée au Figaro mercredi...,"Les médecins jugés ""gros prescripteurs d'arrêt...",Dans une interview accordée au Figaro mercredi...
4,Le préjudice est estimé à 2 millions d'euros. ...,Il aura fallu mobiliser 90 gendarmes pour cett...,Le préjudice est estimé à 2 millions d'euros. ...
...,...,...,...
21396,"Une ""main tendue"" que refuse de saisir l'eurod...",POLITIQUE. La présidente du Front national a s...,"Une ""main tendue"" que refuse de saisir l'eurod..."
21397,"Venue présenter son dernier spectacle, Elektri...",Que serait un dimanche matin sans un passage e...,"Venue présenter son dernier spectacle, Électri..."
21398,Mauvaise nouvelle pour les amateurs de chasse ...,"La chasse à la glu sera interdite cette année,...",Mauvaise nouvelle pour les amateurs de chasse ...
21399,"""Ce n'est pas une question de simple voile, c'...","Bruno Le Maire, candidat à la primaire de la d...","""Ce n'est pas une question de simple voile, c'..."


In [ ]:
df_train_grammar['length_difference'] = df_train_grammar['text_grammar'].str.len() - df_train_grammar['text'].str.len()

In [ ]:
df_train_grammar

,text,titles,text_grammar,length_difference
0,Thierry Mariani sur la liste du Rassemblement ...,L'information n'a pas été confirmée par l'inté...,Thierry Marianne sur la liste du Rassemblement...,8
1,C'est désormais officiel : Alain Juppé n'est p...,Le maire de Bordeaux ne fait plus partie des R...,C'est désormais officiel : Alain Juppé n'est p...,0
2,La mesure est décriée par les avocats et les m...,"En 2020, les tribunaux d'instance fusionnent a...",La mesure est décriée par les avocats et les m...,0
3,Dans une interview accordée au Figaro mercredi...,"Les médecins jugés ""gros prescripteurs d'arrêt...",Dans une interview accordée au Figaro mercredi...,-4
4,Le préjudice est estimé à 2 millions d'euros. ...,Il aura fallu mobiliser 90 gendarmes pour cett...,Le préjudice est estimé à 2 millions d'euros. ...,4
...,...,...,...,...
21396,"Une ""main tendue"" que refuse de saisir l'eurod...",POLITIQUE. La présidente du Front national a s...,"Une ""main tendue"" que refuse de saisir l'eurod...",9
21397,"Venue présenter son dernier spectacle, Elektri...",Que serait un dimanche matin sans un passage e...,"Venue présenter son dernier spectacle, Électri...",6
21398,Mauvaise nouvelle pour les amateurs de chasse ...,"La chasse à la glu sera interdite cette année,...",Mauvaise nouvelle pour les amateurs de chasse ...,0
21399,"""Ce n'est pas une question de simple voile, c'...","Bruno Le Maire, candidat à la primaire de la d...","""Ce n'est pas une question de simple voile, c'...",3


In [ ]:
df_train_grammar['text_grammar'].str.len()

0        1851
1        1368
2        4430
3        2594
4        2116
         ... 
21396    1757
21397    1081
21398    1560
21399    1243
21400    3552
Name: text_grammar, Length: 21401, dtype: int64

In [ ]:
import difflib

idx = [10, 50, 2580, 10580]

for index, row in df_train_grammar.iloc[idx].iterrows():
    print(row['text'])
    print(row['text_grammar'])
    print("\n")

Après une septième édition impressionnante, Danse avec les stars a confirmé son statut de programme incontournable dans le paysage audiovisuel français actuel. Avec des chorégraphies millimétrées, une production classieuse, des candidats survoltés et un jury de professionnels passionné, TF1 offre chaque semaine aux fidèles de l'émission une représentation exceptionnelle. Mais si la prochaine année du concours était celle du changement ? En effet, il se pourrait bien qu'un pilier du show ne rempile pas pour la saison 8...Un membre incontournableEt ce n'est autre que l'une des juges qui vient d'émettre des réserves pour noter les futures célébrités qui fouleront le dance-floor de DALS ! Marie-Claude Pietragalla a en effet révélé que son retour était probablement compromis, ce qui ne manque pas de décevoir ses fans. Bien qu'elle ne soit pas un élément historique de cette immense locomotive, elle répond néanmoins présente à l'appel depuis 2012, gratifiant les participants de ses conseils p

### `c.3.` Conclusions

Using `import language_tool_python` yields effective results in grammar correction for the training set. Notable outcomes include:

* data i = 10 - add space<br>
  - <i>Before</i>: Un membre incontournableEt ce n'est autre<br>
  - <i>After</i>: Un membre incontournable Et ce n'est autre<br>

* data i = 50 - change words<br>
  - <i>Before</i>: jusqu'à ce que les Etats remplissent certaines conditions en matière notamment de respect des droits humains.<br>
  - <i>After</i>: jusqu'à ce que les États remplissent certaines conditions en matière particulièrement de respect des droits humains.<br>

* data i = 2580 - a little wrong "du" -> "de la"<br>
  - <i>Before</i>: Les trois personnes atteintes du Covid-19 sont prises en charge au centre hospitalier d'Ajaccio<br>
  - <i>After</i>: Les trois personnes atteintes de la Covid-19 sont prises en charge au centre hospitalier d'Ajaccio<br>

* data i = 10580 - cange to synonyms<br>
  - <i>Before</i>: Leur analyse sur l'échec du parti aux élections législatives est également requis<br>
  - <i>After</i>: Leur analyse sur l'échec du parti aux élections législatives est aussi requis<br>

### `c.4.` Applying the function to both the validation set and the test set.

In [ ]:
## VALIDATION
df_validation = pd.read_csv('/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/validation.csv')

# Apply grammar correction to all news articles in the DataFrame
tqdm.pandas()
df_validation['text_grammar'] = df_validation['text'].progress_apply(correct_grammar)
file_path = "/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/validation_grammar_correction.csv"
df_validation.to_csv(file_path, index=False)
print("File saved to Google Drive at:", file_path)

100%|██████████| 1500/1500 [20:28<00:00,  1.22it/s]


File saved to Google Drive at: /content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/validation_grammar_correction.csv


In [ ]:
## TESTING
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/test_text.csv')

# Apply grammar correction to all news articles in the DataFrame
tqdm.pandas()
df_test['text_grammar'] = df_test['text'].progress_apply(correct_grammar)
file_path = "/content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/text_grammar_correction.csv"
df_test.to_csv(file_path, index=False)
print("File saved to Google Drive at:", file_path)

100%|██████████| 1500/1500 [21:11<00:00,  1.18it/s]


File saved to Google Drive at: /content/drive/My Drive/Colab Notebooks/11_DataAI_INF582_TextMining_NLP/Challenge/data/processed/text_grammar_correction.csv
